**Twitter Elections Jake**<br>
*Compsci 209 Final Project*

Follow the instructions here for pulling tweets:<br>
https://developer.twitter.com/en/docs/tweets/search/api-reference/premium-search#DataEndpoint

----